# 2D *tobac* Tutorial: GOES-16 ABI Data

This tutorial will demonstrate how to use *tobac* to detect and track cloud features with GOES-16 ABI IR data. Because this tutorial is entirely using 2D spatial data, it only requires *tobac* v1.3.1 or newer, rather than the beta of v1.5. 

In [ ]:
%matplotlib inline
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import goes_conv
import cartopy.crs as ccrs
import matplotlib.gridspec as gridspec
import datetime
import pandas as pd
import cartopy.feature as cfeature
import tobac
import s3fs

### Data Prep

First, we will start with a single time/file. Then we will demonstrate expanding this to multiple times. 

In [ ]:
# Download example file 1 from here: https://noaa-goes16.s3.amazonaws.com/ABI-L2-MCMIPC/2021/178/14/OR_ABI-L2-MCMIPC-M6_G16_s20211781401171_e20211781403549_c20211781404049.nc
fs = s3fs.S3FileSystem(anon=True)
aws_url = 's3://noaa-goes16/ABI-L2-MCMIPC/2021/178/14/OR_ABI-L2-MCMIPC-M6_G16_s20211781401171_e20211781403549_c20211781404049.nc'
#in_goes_test = xr.open_dataset('./OR_ABI-L2-MCMIPC-M6_G16_s20211781401171_e20211781403549_c20211781404049.nc')
with fs.open(aws_url) as fileObj:
  in_goes_test = xr.open_dataset(fileObj, engine='h5netcdf')


In [ ]:
in_goes_test

GOES data comes in radian coordinates from the satellite. There are many different ways of gridding these data, but we've included a little helper function here that should work. 

In [ ]:
llons, llats  = goes_conv.lat_lon_reproj(in_goes_test)

Since we are only interested in one channel here, let's extract out those data. This will also make our lives easier when switching over to Iris.

In [ ]:
in_goes_test_for_tobac = xr.Dataset(data_vars=
    {
        "C10": (("time", "Y", "X"), np.expand_dims(in_goes_test['CMI_C10'].values, axis=0)),
    },
    coords=
    {"time": [in_goes_test['time_bounds'].mean().values],
    "lon": (["Y", "X"], llons),
    "lat": (["Y", "X"], llats)},
)

*tobac* currently only supports Iris as its data input type. Fortunately, it is relatively easy to convert Xarray to Iris.

In [ ]:
iris_var = in_goes_test_for_tobac['C10'].to_iris()

In [ ]:
iris_var

### Feature Detection

First, let's set our parameters for feature detection. 

In [ ]:
# Keyword arguments for the feature detection step
# See: https://tobac--150.org.readthedocs.build/en/150/threshold_detection_parameters.html for a more thorough description of these parameters.
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=1.5
parameters_features['n_erosion_threshold']=2

parameters_features['n_min_threshold']=10
parameters_features['target']='minimum'
parameters_features['threshold']=[230, 220, 210, 200]


Let's run feature detection. While this is only running on one file, 

In [ ]:
print('starting feature detection')
# GOES-16 isn't *quite* 2km resolution, but this is reasonably close. 
dxy=2000
Features=tobac.feature_detection_multithreshold(iris_var,dxy,**parameters_features)

In [ ]:
# This is our list of detected features. 
Features

In [ ]:
#get cartopy projection for plotting parts of figure
proj = ccrs.PlateCarree()


In [ ]:
goes_xformed_pts = proj.transform_points(src_crs = ccrs.PlateCarree(), x=llons, y=llats)
goes_proj_x = goes_xformed_pts[...,0]
goes_proj_y = goes_xformed_pts[...,1]


In [ ]:
#make the figure
fig1 = plt.figure(figsize = (14,14), constrained_layout=True)
spec1 = gridspec.GridSpec(ncols=1, nrows=1, figure=fig1)

f1_goes_1 = fig1.add_subplot(spec1[0, 0], projection=proj)
#f1_goes_1.gridlines(xlocs=xticks, ylocs=yticks, linestyle=':', zorder=3)
# Label the end-points of the gridlines using the custom tick makers:
#f1_goes_1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
#f1_goes_1.yaxis.set_major_formatter(LATITUDE_FORMATTER)
f1_goes_1.coastlines(resolution='50m')
#f1_goes_1.borders(resolution='50m')
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
country_borders = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')

f1_goes_1.add_feature(states_provinces, edgecolor='gray')
f1_goes_1.add_feature(country_borders, edgecolor='gray')

#this works for squares onlu, figure out non-squares.
min_x = 400
min_y = 100
max_x = 2500
max_y = 1100


cmesh = f1_goes_1.pcolormesh(goes_proj_x[min_y:max_y, min_x:max_x].T, goes_proj_y[min_y:max_y, min_x:max_x].T, in_goes_test['CMI_C10'][min_y:max_y, min_x:max_x].T)
plt.colorbar(cmesh)

#f1_goes_1.scatter(goes_proj_x[min_y:max_y, min_x:max_x], goes_proj_y[min_y:max_y, min_x:max_x], )
f1_goes_1.scatter(Features['lon'],Features['lat'], transform=ccrs.PlateCarree(), color='r')
f1_goes_1.set_extent([-100, -65, 23, 45])

### Segmentation

Segmentation associates each feature with an area of another variable around it that is either the same variable or a different variable.

First, we will try to associate the areas of the same variable with a minimum threshold of 230K. 

In [ ]:
parameters_segmentation = dict()
parameters_segmentation['threshold'] = 230
parameters_segmentation['target'] = 'minimum'

In [ ]:
print('starting segmentation')
dxy=2000
segmentation, features_from_seg=tobac.segmentation.segmentation(Features, iris_var, dxy,**parameters_segmentation)

In [ ]:
#make the figure
fig1 = plt.figure(figsize = (14,14), constrained_layout=True)
spec1 = gridspec.GridSpec(ncols=1, nrows=1, figure=fig1)

f1_goes_1 = fig1.add_subplot(spec1[0, 0], projection=proj)
#f1_goes_1.gridlines(xlocs=xticks, ylocs=yticks, linestyle=':', zorder=3)
# Label the end-points of the gridlines using the custom tick makers:
#f1_goes_1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
#f1_goes_1.yaxis.set_major_formatter(LATITUDE_FORMATTER)
f1_goes_1.coastlines(resolution='50m')
#f1_goes_1.borders(resolution='50m')
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
country_borders = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')

f1_goes_1.add_feature(states_provinces, edgecolor='gray')
f1_goes_1.add_feature(country_borders, edgecolor='gray')

#this works for squares onlu, figure out non-squares.
min_x = 400
min_y = 100
max_x = 2500
max_y = 1100


cmesh = f1_goes_1.pcolormesh(goes_proj_x[min_y:max_y, min_x:max_x].T, goes_proj_y[min_y:max_y, min_x:max_x].T, in_goes_test['CMI_C10'][min_y:max_y, min_x:max_x].T)
segmentation_data = segmentation.core_data()[min_y:max_y, min_x:max_x].T
# Loop through each segmented area to draw each one individually
for unique_val in np.unique(segmentation_data):
        if unique_val == 0:
                continue
        curr_seg_mask = segmentation_data == unique_val
        curr_seg_mask = curr_seg_mask.astype(np.int32)
        cl = f1_goes_1.contour(goes_proj_x[min_y:max_y, min_x:max_x].T, goes_proj_y[min_y:max_y, min_x:max_x].T, curr_seg_mask, levels=[1,], colors='r')
plt.colorbar(cmesh)

#f1_goes_1.scatter(goes_proj_x[min_y:max_y, min_x:max_x], goes_proj_y[min_y:max_y, min_x:max_x], )
f1_goes_1.scatter(Features['lon'],Features['lat'], transform=ccrs.PlateCarree(), color='r')
f1_goes_1.set_extent([-100, -65, 23, 45])

#### Segmentation on a different variable, same grid

In [ ]:
# Let's try to segment on visible satellite from our IR detected features.
in_goes_test_for_tobac_vis = xr.Dataset(data_vars=
    {
        "C02": (("time", "Y", "X"), np.expand_dims(in_goes_test['CMI_C02'].values, axis=0)),
    },
    coords=
    {"time": [in_goes_test['time_bounds'].mean().values],
    "lon": (["Y", "X"], llons),
    "lat": (["Y", "X"], llats)},
)

In [ ]:
iris_var_vis = in_goes_test_for_tobac_vis['C02'].to_iris()

In [ ]:
parameters_segmentation = dict()
parameters_segmentation['threshold'] = 0.4
parameters_segmentation['target'] = 'maximum'

In [ ]:
print('starting segmentation on visible satellite')
dxy=2000
segmentation, features_from_seg=tobac.segmentation.segmentation(Features, iris_var_vis, dxy,**parameters_segmentation)

In [ ]:
#make the figure
fig1 = plt.figure(figsize = (14,14), constrained_layout=True)
spec1 = gridspec.GridSpec(ncols=1, nrows=1, figure=fig1)

f1_goes_1 = fig1.add_subplot(spec1[0, 0], projection=proj)
#f1_goes_1.gridlines(xlocs=xticks, ylocs=yticks, linestyle=':', zorder=3)
# Label the end-points of the gridlines using the custom tick makers:
#f1_goes_1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
#f1_goes_1.yaxis.set_major_formatter(LATITUDE_FORMATTER)
f1_goes_1.coastlines(resolution='50m')
#f1_goes_1.borders(resolution='50m')
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
country_borders = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')

f1_goes_1.add_feature(states_provinces, edgecolor='gray')
f1_goes_1.add_feature(country_borders, edgecolor='gray')

#this works for squares onlu, figure out non-squares.
min_x = 400
min_y = 100
max_x = 2500
max_y = 1100


cmesh = f1_goes_1.pcolormesh(goes_proj_x[min_y:max_y, min_x:max_x].T, goes_proj_y[min_y:max_y, min_x:max_x].T, in_goes_test['CMI_C02'][min_y:max_y, min_x:max_x].T, cmap='binary_r')
segmentation_data = segmentation.core_data()[min_y:max_y, min_x:max_x].T
# Loop through each segmented area to draw each one individually
for unique_val in np.unique(segmentation_data):
        if unique_val == 0:
                continue
        curr_seg_mask = segmentation_data == unique_val
        curr_seg_mask = curr_seg_mask.astype(np.int32)
        cl = f1_goes_1.contour(goes_proj_x[min_y:max_y, min_x:max_x].T, goes_proj_y[min_y:max_y, min_x:max_x].T, curr_seg_mask, levels=[1,], colors='r')
plt.colorbar(cmesh)

#f1_goes_1.scatter(goes_proj_x[min_y:max_y, min_x:max_x], goes_proj_y[min_y:max_y, min_x:max_x], )
f1_goes_1.scatter(Features['lon'],Features['lat'], transform=ccrs.PlateCarree(), color='r')
f1_goes_1.set_extent([-100, -65, 23, 45])

## Tracking

For tracking, we will clearly need more than one timestep. Now, let's load in the 

In [ ]:
import urllib.request
in_file_names = ['OR_ABI-L2-MCMIPC-M6_G16_s20211781401171_e20211781403549_c20211781404049.nc', 'OR_ABI-L2-MCMIPC-M6_G16_s20211781406171_e20211781408549_c20211781409055.nc',
                'OR_ABI-L2-MCMIPC-M6_G16_s20211781411171_e20211781413544_c20211781414055.nc', 'OR_ABI-L2-MCMIPC-M6_G16_s20211781416171_e20211781418555_c20211781419046.nc',
                'OR_ABI-L2-MCMIPC-M6_G16_s20211781421171_e20211781423544_c20211781424047.nc', 'OR_ABI-L2-MCMIPC-M6_G16_s20211781426171_e20211781428544_c20211781429045.nc',
                'OR_ABI-L2-MCMIPC-M6_G16_s20211781431171_e20211781433544_c20211781434046.nc', 'OR_ABI-L2-MCMIPC-M6_G16_s20211781436171_e20211781438555_c20211781439046.nc',
                'OR_ABI-L2-MCMIPC-M6_G16_s20211781441171_e20211781443549_c20211781444049.nc', 'OR_ABI-L2-MCMIPC-M6_G16_s20211781446171_e20211781448544_c20211781449043.nc',
                'OR_ABI-L2-MCMIPC-M6_G16_s20211781451171_e20211781453544_c20211781454048.nc', 'OR_ABI-L2-MCMIPC-M6_G16_s20211781456171_e20211781458550_c20211781459045.nc']
for in_file in in_file_names:
    print("Downloading "+in_file)
    urllib.request.urlretrieve("https://noaa-goes16.s3.amazonaws.com/ABI-L2-MCMIPC/2021/178/14/"+in_file, "./goes_data/"+in_file)


In [ ]:
import glob
# Import all of our data into an xarray
in_goes_files = sorted(glob.glob('./goes_data/*.nc', recursive=True))

We need to combine these GOES-16 files into one large Xarray dataset. 

In [ ]:
in_goes_ds = list()

for in_goes_fn in in_goes_files:
    #print(in_goes_fn)
    in_goes_ds.append(xr.open_dataset(in_goes_fn))
    
#get common lats/lons

llons, llats  = goes_conv.lat_lon_reproj(in_goes_ds[0])

full_goes_data = np.zeros((len(in_goes_ds),llons.shape[0], llons.shape[1]))
goes_times = list()

for i, in_goes_file in enumerate(in_goes_ds):
    full_goes_data[i] = in_goes_file['CMI_C10'].values
    goes_times.append(pd.to_datetime(in_goes_file['time_bounds'].mean().values).to_pydatetime())
#pd.to_datetime(test_dt).to_pydatetime()
in_goes_for_tobac = xr.Dataset(data_vars=
    {
        "C10": (("time", "Y", "X"), full_goes_data,)
    },
    coords=
    {"time": goes_times,
    "lon": (["Y", "X"], llons),
    "lat": (["Y", "X"], llats)},
)


As before, we will convert our input data to Iris

In [ ]:
iris_var = in_goes_for_tobac['C10'].to_iris()

In [ ]:
# Keyword arguments for the feature detection step (same as before)
# See: https://tobac--150.org.readthedocs.build/en/150/threshold_detection_parameters.html for a more thorough description of these parameters.
parameters_features={}
parameters_features['position_threshold']='weighted_diff'
parameters_features['sigma_threshold']=1.5
parameters_features['n_erosion_threshold']=2

parameters_features['n_min_threshold']=10
parameters_features['target']='minimum'
parameters_features['threshold']=[230, 220, 210, 200]


In [ ]:
# Feature detection and save results to file:
print('starting feature detection')
dxy=2000
Features=tobac.feature_detection_multithreshold(iris_var,dxy,**parameters_features)

In [ ]:
Features

In [ ]:
parameters_linking={}
parameters_linking['method_linking']='predict'
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['extrapolate']=0
parameters_linking['order']=1
parameters_linking['subnetwork_size']=200
parameters_linking['memory']=2
parameters_linking['time_cell_min']=5*60
parameters_linking['method_linking']='predict'
parameters_linking['v_max']=30


In [ ]:
# Perform linking and save results:
Track= tobac.linking_trackpy(Features,None,dt=5*60,dxy=1000,**parameters_linking)


In [ ]:
Track

In [ ]:
#make the figure
fig1 = plt.figure(figsize = (14,14), constrained_layout=True)
spec1 = gridspec.GridSpec(ncols=1, nrows=1, figure=fig1)

f1_goes_1 = fig1.add_subplot(spec1[0, 0], projection=proj)
#f1_goes_1.gridlines(xlocs=xticks, ylocs=yticks, linestyle=':', zorder=3)
# Label the end-points of the gridlines using the custom tick makers:
#f1_goes_1.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
#f1_goes_1.yaxis.set_major_formatter(LATITUDE_FORMATTER)
f1_goes_1.coastlines(resolution='50m')
#f1_goes_1.borders(resolution='50m')
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
country_borders = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_0_countries',
        scale='50m',
        facecolor='none')

f1_goes_1.add_feature(states_provinces, edgecolor='gray')
f1_goes_1.add_feature(country_borders, edgecolor='gray')

#this works for squares onlu, figure out non-squares.
min_x = 400
min_y = 100
max_x = 2500
max_y = 1100


cmesh = f1_goes_1.pcolormesh(goes_proj_x[min_y:max_y, min_x:max_x].T, goes_proj_y[min_y:max_y, min_x:max_x].T, in_goes_for_tobac['C10'][-1,min_y:max_y, min_x:max_x].T)
plt.colorbar(cmesh)

curr_goes_time = Features['time'].max()
features_at_time = Track[Track['time']== curr_goes_time]
cells_at_time = features_at_time[features_at_time['cell']!=-1]
#f1_goes_1.scatter(goes_proj_x[min_y:max_y, min_x:max_x], goes_proj_y[min_y:max_y, min_x:max_x], )
f1_goes_1.scatter(features_at_time['lon'],features_at_time['lat'], transform=ccrs.PlateCarree(), color='r')
track_before_time = Track[np.logical_and(Track['cell'].isin(np.unique(cells_at_time['cell'])),
                                                Track['time']<=curr_goes_time+datetime.timedelta(seconds=5))]
track_lats = track_before_time.groupby('cell')['lat'].apply(list)
track_lons = track_before_time.groupby('cell')['lon'].apply(list)

for track_lat_list, track_lon_list in zip(track_lats.values, track_lons.values):
        f1_goes_1.plot(np.array(track_lon_list).flatten(), 
                        np.array(track_lat_list).flatten(), transform=ccrs.PlateCarree(), color='r')
f1_goes_1.set_extent([-100, -65, 23, 45])